In [16]:
import boto3
import json, uuid, copy, datetime
import random, names, tabulate

In [17]:
# open_cmd_in_new_terminal("docker-compose up")
!gnome-terminal -- docker-compose up

In [18]:
!./setuprds.sh
!./setupdynamo.sh

Setting up users
psql:../templates/rds/create_db_roles.sql:3: ERROR:  role "account_api_worker" already exists
psql:../templates/rds/create_db_roles.sql:4: ERROR:  role "save_tx_api_worker" already exists
psql:../templates/rds/create_db_roles.sql:5: ERROR:  role "float_api_worker" already exists
psql:../templates/rds/create_db_roles.sql:7: ERROR:  role "auth_api_worker" already exists
Setting up account ledger in RDS local
psql:../templates/rds/create_account_ledger.sql:1: NOTICE:  schema "account_data" already exists, skipping
CREATE SCHEMA
psql:../templates/rds/create_account_ledger.sql:20: NOTICE:  relation "core_account_ledger" already exists, skipping
CREATE TABLE
psql:../templates/rds/create_account_ledger.sql:22: ERROR:  relation "owner_id_idx" already exists
psql:../templates/rds/create_account_ledger.sql:23: ERROR:  relation "opening_user_idx" already exists
REVOKE
GRANT
GRANT
GRANT
GRANT
GRANT
GRANT
Setting up transaction ledger in RDS local
psql:../templates/rds/create_trans

In [19]:
!./deploylambdas.sh

Deploying the float lambda
Serverless: config.options_stage: local
Serverless: serverless.service.custom.stage: undefined
Serverless: serverless.service.provider.stage: dev
Serverless: config.stage: local
Serverless: Using serverless-localstack
Serverless: Reconfiguring service apigateway to use http://localhost:4567
Serverless: Reconfiguring service cloudformation to use http://localhost:4581
Serverless: Reconfiguring service cloudwatch to use http://localhost:4582
Serverless: Reconfiguring service lambda to use http://localhost:4574
Serverless: Reconfiguring service dynamodb to use http://localhost:4569
Serverless: Reconfiguring service kinesis to use http://localhost:4568
Serverless: Reconfiguring service route53 to use http://localhost:4580
Serverless: Reconfiguring service firehose to use http://localhost:4573
Serverless: Reconfiguring service stepfunctions to use http://localhost:4585
Serverless: Reconfiguring service es to use http://localhost:4578
Serverless: Reconfiguring serv

In [20]:
dynamodb = boto3.client('dynamodb', endpoint_url = 'http://localhost:4569')
local_lambda = boto3.client('lambda', endpoint_url = 'http://localhost:4574')

In [21]:
def print_deployed_functions():
    function_list = local_lambda.list_functions()
    # print(function_list)
    print('Function list: ', [function['FunctionName'] for function in function_list['Functions']])

In [22]:
def generate_account(client_id = 'zar_client_co', float_id = 'zar_cash_float'):
    first_name = names.get_first_name()
    family_name = names.get_last_name()
    user_id = str(uuid.uuid4())
    return { 'clientId': client_id, 'floatId': float_id, 'ownerUserId': user_id, 'userFirstName': first_name, 'userFamilyName': family_name}

In [23]:
def decode_lambda_result(lambda_result):
    lambda_payload = lambda_result['Payload'].read()
    lambda_pload_decoded = lambda_payload.decode('utf-8')
    lambda_pload_object = json.loads(lambda_pload_decoded)
    return lambda_pload_object

In [24]:
def create_number_accounts(number_accounts = 1, client_id = 'zar_client_co'):
    account_dicts = [generate_account(client_id) for i in range(number_accounts)]
    persisted_accounts = []
    for account in account_dicts:
        lambda_result = local_lambda.invoke(FunctionName='create-account', InvocationType='RequestResponse', 
                                           Payload=json.dumps(account))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_account = copy.deepcopy(account)
        persisted_account['accountId'] = lambda_payload['accountId']
        persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Account with ID %s persisted at %s' % (persisted_account['accountId'], persisted_account['persistedTime']))
        persisted_accounts.append(persisted_account)
    
    return persisted_accounts
    

In [25]:
def generate_saving_transaction(account_id, ref_amount = 100, float_id = 'zar_cash_primary'):
    current_time = datetime.datetime.now()
    saved_amount = round(random.random() * ref_amount * 10 * 10) # random proportion of ref amount, 
    test_saving_dict = { 
        'accountId': account_id, 
        'initiationTime': str(current_time), 
        'settlementTime': str(current_time), 
        'savedAmount': saved_amount, 
        'savedCurrency': 'ZAR', 
        'savedUnit': 'HUNDREDTH_CENT',
        'floatId': float_id
    }
    return test_saving_dict

In [26]:
def seed_savings_for_accounts(accounts, tx_per_account = 1, base_amount = 100):
    transactions = []
    for i in range(tx_per_account):
        transactions.extend([generate_saving_transaction(account['accountId'], base_amount) for account in accounts])
#     print('transactions: ', transactions)
    tx_records = []
    for tx in transactions:
        lambda_result = local_lambda.invoke(FunctionName='add-savings', InvocationType='RequestResponse', 
                                           Payload=json.dumps(tx, default=str))
        lambda_payload = json.loads(decode_lambda_result(lambda_result)['body'])
        persisted_tx = copy.deepcopy(tx)
        persisted_tx['transactionId'] = lambda_payload['transactionDetails'][0][0]['transaction_id']
        persisted_tx['currentBalance'] = lambda_payload['newBalance']
        # persisted_account['persistedTime'] = lambda_payload['persistedTime']
        print('Transaction persisted with ID %s' % (persisted_tx['transactionId']))
        tx_records.append(persisted_tx)
    
    return tx_records

In [27]:
print_deployed_functions()

Function list:  ['float-api-local-accrue']


In [28]:
accounts = create_number_accounts(number_accounts = 20)

ResourceNotFoundException: An error occurred (ResourceNotFoundException) when calling the Invoke operation: Function does not exist: arn:aws:lambda:eu-west-1:000000000000:function:create-account

In [ ]:
account_info = [{ 'Name': account['userFirstName'] + ' ' + account['userFamilyName'], 
                'AccountId': account['accountId'] } for account in accounts]
print(tabulate.tabulate(account_info, headers = 'keys'))

In [ ]:
single_tx = generate_saving_transaction(accounts[0]['accountId'], 100)
lambda_result = local_lambda.invoke(FunctionName='add-savings', Payload=json.dumps(single_tx, default=str))
result_decoded = decode_lambda_result(lambda_result)
print('Lambda result', result_decoded)

In [ ]:
account_balances = seed_savings_for_accounts(accounts, 1, 100)

In [ ]:
print([account['currentBalance'] for account in account_balances])